# Start a chat with LLM

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- get AI model inferencing with `Services`
- enable effortless multi-round exchange between many participants. 

A Session object contains one or more `branch`, by default, every session has a `'main'` branch， `session.branches['main']`

- `branch.messages` is a `pd.DataFrame`

lionagi has some default config, you can check them under lionagi.integrations.config

In [1]:
from IPython.display import Markdown  # for better print

In [2]:
from lionagi.integrations.config import oai_schema

oai_schema["chat/completions"]["config"]

{'model': 'gpt-4-turbo',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.7,
 'top_p': 1,
 'tools': None,
 'tool_choice': 'none',
 'user': None}

In [3]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = (
    "very short joke: a blue whale and a big shark meet at the bar and start dancing"
)

In [4]:
from lionagi import Session

# create a llm conversation with OpenAI gpt-4 (default)
comedian1 = Session(system=sys_comedian)
joke1 = await comedian1.chat(instruction=instruct1, max_token=50)

In [5]:
Markdown(joke1)

Why did the blue whale and the big shark start dancing at the bar? Because the jukebox only played sea-shanties, and they couldn’t find their porpoise in life!

You can change a LLM provider by choosing a pre-configured service, or create your own

In [6]:
from lionagi import Services

service = Services.OpenRouter(
    max_tokens=1000, max_requests=10, interval=60
)  # set tokens rate limit
model = "mistralai/mixtral-8x7b-instruct"  # use any model from openrouter, you need an openrouter api key

comedian2 = Session(system=sys_comedian, service=service)
joke2 = await comedian2.chat(instruction=instruct1, model=model, max_token=50)

In [7]:
Markdown(joke2)

 Sure thing, here's a short and sarcastically funny joke for you:

"A blue whale and a big shark walk into a bar and start cutting a rug on the dance floor. I mean, who knew these two could move so gracefully, right? I guess when it comes to dancing, size doesn't matter. Ha! Just kidding, it totally does. Those two were taking up the whole dance floor."

In [8]:
# The following three implementation are equivalent
# for getting the messages DataFrame from the default branch

"""
comedian2.messages
comedian2.default_branch.messages
comedian2.branches['main'].messages
"""

comedian2.branches["main"].messages

,node_id,timestamp,role,sender,recipient,content
0,97729ba0aea82c9d7f09d3417733abb6,2024-04-21T00:12:49.152361+00:00,system,system,assistant,"{""system_info"": ""As a comedian, you are sarcas..."
1,000b1b557be5e2777b26f08a2c9f7316,2024-04-21T00:12:49.152958+00:00,user,user,main,"{""instruction"": ""very short joke: a blue whale..."
2,cfe7576cbe1c423808f5c41de15938d3,2024-04-21T00:12:51.819032+00:00,assistant,main,user,"{""response"": "" Sure thing, here's a short and ..."


In [9]:
# check the default branch information
comedian2.default_branch.info

{'system': 1, 'user': 1, 'assistant': 1, 'total': 3}